# Normal Numpy script

In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from profiling import get_memory, get_time
from plotting import plot_memory, plot_time
from cosine_simiilarity_top_k import cosine_similarity_top_k, get_memory_available

In [2]:
numba_function_kwargs = {"top_k": 100, "max_memory_bytes": 10E9}
get_memory(cosine_similarity_top_k, n_items=int(1E4), embedding_size=100, function_kwargs=numba_function_kwargs)

Memory available: 23.37 GB
Using usage: 10.00 GB
Number of threads: 16
Chunk size per thread: 3893


(1649134493, 16080008)

In [3]:
get_memory_available() / 1E9

24.494534656

In [4]:
import math
M = 20E9
math.sqrt(M / 8)

50000.0

In [8]:
embedding_size = 100
step = int(5E3)
max_n_items = int(5E4)
n_items = range(step, max_n_items + 1, step)
max_memory_bytes = int(5E9)
top_k = 100
numba_function_kwargs = {"top_k": top_k, "max_memory_bytes": max_memory_bytes, "force_memory": True}

max_memory = {
    "numpy": [],
    "numba": []
}
matrix_memory = {
    "numpy": [],
    "numba": []
}
execution_time = {
    "numpy": [],
    "numba": []
}
for n in n_items:
    print(f"\n\nN: {n}")
    print("Sklean: memory use")
    if n > 50000:
        max_size, matrix_size = None, None
    else:
        max_size, matrix_size = get_memory(cosine_similarity, n_items=n, embedding_size=embedding_size)
    print("\nSklean: execution time")
    if n > 50000:
        _time = None
    else:
        _time = get_time(cosine_similarity, n_items=n, embedding_size=embedding_size, n_iterations=1)
    max_memory["numpy"].append(max_size), matrix_memory["numpy"].append(matrix_size), execution_time["numpy"].append(_time)
    
    print("\nCustom: memory use")
    max_size, matrix_size = get_memory(cosine_similarity_top_k, n_items=n, embedding_size=embedding_size, function_kwargs=numba_function_kwargs)
    print("\nCustom: execution time")
    _time = get_time(cosine_similarity_top_k, n_items=n, embedding_size=embedding_size, function_kwargs=numba_function_kwargs, n_iterations=1)
    max_memory["numba"].append(max_size), matrix_memory["numba"].append(matrix_size), execution_time["numba"].append(_time)



N: 5000
Sklean: memory use

Sklean: execution time

Custom: memory use
Memory available: 23.12 GB
Using usage: 5.00 GB
Number of threads: 16
Chunk size per thread: 3893

Custom: execution time
Memory available: 23.03 GB
Using usage: 5.00 GB
Number of threads: 16
Chunk size per thread: 3893


N: 10000
Sklean: memory use

Sklean: execution time

Custom: memory use
Memory available: 23.01 GB
Using usage: 5.00 GB
Number of threads: 16
Chunk size per thread: 1940

Custom: execution time
Memory available: 21.87 GB
Using usage: 5.00 GB
Number of threads: 16
Chunk size per thread: 1940


N: 15000
Sklean: memory use

Sklean: execution time

Custom: memory use
Memory available: 21.94 GB
Using usage: 5.00 GB
Number of threads: 16
Chunk size per thread: 1289

Custom: execution time
Memory available: 20.83 GB
Using usage: 5.00 GB
Number of threads: 16
Chunk size per thread: 1289


N: 20000
Sklean: memory use

Sklean: execution time

Custom: memory use
Memory available: 20.80 GB
Using usage: 5.00 

In [9]:
import pandas as pd

dfs = {
    "max_memory (GB)": pd.DataFrame.from_dict(max_memory) / 1E9,
    "matrix_memory (GB)": pd.DataFrame.from_dict(matrix_memory) / 1E9,
    "execution_time (s)": pd.DataFrame.from_dict(execution_time),
}
df = pd.concat(dfs.values(), keys=dfs.keys(), axis=1)
df["n_items"] = n_items
df["embedding_size"] = embedding_size
df["max_memory_bytes (GGB)"] = int(max_memory_bytes / 1E9)
df["top_k"] = top_k
df.set_index("n_items", inplace=True)
df

max_memory (GB)           matrix_memory (GB)           \
                  numpy     numba              numpy    numba   
n_items                                                         
5000           0.204001  0.410739                0.2  0.00804   
10000          0.808001  1.573140                0.8  0.01608   
15000          1.812004  3.628420                1.8  0.02412   
20000          3.216004  4.740562                3.2  0.03216   
25000          5.020004  4.962281                5.0  0.04020   
30000          7.224004  4.955651                7.2  0.04824   
35000          9.828004  4.488100                9.8  0.05628   
40000         12.832004  4.327168               12.8  0.06432   
45000         16.236004  4.778869               16.2  0.07236   
50000         20.040004  4.322808               20.0  0.08040   

        execution_time (s)            embedding_size max_memory_bytes (GGB)  \
                     numpy      numba                                         
n_items                                                                       
5000              0.122580   0.184529            100                      5   
10000             0.625890   0.433022            100                      5   
15000             1.662348   0.765893            100                      5   
20000             3.019485   1.289468            100                      5   
25000             4.708843   1.950410            100                      5   
30000             7.096138   2.829469            100                      5   
35000            10.443496   3.375207            100                      5   
40000            15.164635   5.832886            100                      5   
45000            18.301724   5.830828            100                      5   
50000            22.924060  10.570714            100                      5   

        top_k  
               
n_items        
5000      100  
10000     100  
15000     100  
20000     100  
25000     100  
30000     100  
35000     100  
40000     100  
45000     100  
50000     100

In [10]:
df.to_csv(f"metrics_embedding_size-{embedding_size}_top_k-{top_k}_max_memory_bytes-{max_memory_bytes}.csv")

In [ ]:
# plot_memory(n_items, max_memory, matrix_memory)
# plt.show()
# plot_time(n_items, execution_time)
# plt.show()

In [ ]:
# from matplotlib import pyplot as plt
# %matplotlib inline

# n_items_big = range(0, int(1E6) + 1, int(1E5))
# plt.figure(facecolor='white')
# plt.scatter(n_items_big, [8 * n**2 / 1E9 for n in n_items_big], color="green", marker="*", label="Memory in GB")
# plt.legend(loc="upper left")
# plt.show()

# plt.figure(facecolor='white')
# coeff_2, coeff_1, coeff_0 = np.polyfit(n_items, execution_time, 2)
# plt.scatter(n_items_big, [(coeff_0 + coeff_1 * n + coeff_2 * n **2) / 3600 for n in n_items_big], color="green", marker="*", label="Hours")
# plt.legend(loc="upper left")
# plt.show()

# coeff_2, coeff_1, coeff_0 = np.polyfit(n_items, execution_time, 2)


$
\begin{bmatrix}
a_1 & a_2 & a_3 \\
b_1 & b_2 & b_3 \\
c_1 & c_2 & c_3 \\
... & ... & ... \\
z_1 & z_2 & z_3 
\end{bmatrix}_{n_{items} \times M}
$

$
\begin{bmatrix}
a_1 & a_2 & a_3 \\
b_1 & b_2 & b_3 \\
c_1 & c_2 & c_3 \\
... & ... & ... \\
z_1 & z_2 & z_3 
\end{bmatrix}_{n_{items} \times M}
\times
\begin{bmatrix}
a_1 & b_1 & c_1 & ... & z_1 \\
a_2 & b_2 & c_2 & ... & z_2 \\
a_3 & b_3 & c_3 & ... & z_3
\end{bmatrix}_{M \times n_{items} }
=
\begin{bmatrix}
a \cdot a & a \cdot b & a \cdot c & ... & a \cdot z \\
b \cdot a & b \cdot b & b \cdot c & ... & b \cdot z \\
c \cdot a & c \cdot b & c \cdot c & ... & c \cdot z \\
... & ... & ... & ... & ... \\
z \cdot a & z \cdot b & z \cdot c & ... & z \cdot z \\
\end{bmatrix}_{n_{items} \times n_{items}}
\leftarrow\text{Similarity Matrix}
$

$
\begin{bmatrix}
a_1 & a_2 & a_3 \\
b_1 & b_2 & b_3 \\
... & ... & ... \\
k_1 & k_2 & k_3 
\end{bmatrix}
\times
\begin{bmatrix}
a_1 & b_1 & c_1 & ... & z_1 \\
a_2 & b_2 & c_2 & ... & z_2 \\
a_3 & b_3 & c_3 & ... & z_3 \\
\end{bmatrix}
=
\begin{bmatrix}
a \cdot a & a \cdot b & a \cdot c & ... & a \cdot z \\
b \cdot a & b \cdot b & b \cdot c & ... & b \cdot z \\
... & ... & ... & ... & ... \\
k \cdot a & k \cdot b & k \cdot c & ... & k \cdot z \\
\end{bmatrix}_{n_{items} \times n_{items}}
\leftarrow\text{Similarity Matrix}
\\
\begin{bmatrix}
k_1 & k_2 & k_3 \\
l_1 & l_2 & l_3 \\
... & ... & ... \\
z_1 & z_2 & z_3 
\end{bmatrix}
\times
\begin{bmatrix}
a_1 & b_1 & c_1 & ... & z_1 \\
a_2 & b_2 & c_2 & ... & z_2 \\
a_3 & b_3 & c_3 & ... & z_3 \\
\end{bmatrix}
=
\begin{bmatrix}
k \cdot a & k \cdot b & k \cdot c & ... & k \cdot z \\
l \cdot a & l \cdot b & l \cdot c & ... & l \cdot z \\
... & ... & ... & ... & ... \\
z \cdot a & z \cdot b & z \cdot c & ... & z \cdot z \\
\end{bmatrix}_{n_{items} \times n_{items}}
\leftarrow\text{Similarity Matrix}
$

In [ ]:
def memory_cosine_similarity_top_k(n_items, embedding_size, top_k, chunk_size, n_threads):
    sparse_matrix_memory = (n_items * top_k) * 2 + n_items  # data + indices + indptr
    per_thread_memory = (chunk_size * n_items) * 2 * n_threads  # (chunk dot product result + argpartition output matrix) x number of threads
    return (sparse_matrix_memory + per_thread_memory) * 8 / 1E9

chunk_size = 2981
n_items = 1E6
embedding_size = 100
top_k = 100
n_threads = 1
memory_cosine_similarity_top_k(n_items, embedding_size, top_k, chunk_size, n_threads)